In [1]:
import sys
sys.path.append('structural-probes')
from pathlib import Path

from run_experiment import setup_new_experiment_dir, execute_experiment
import yaml
import torch
import pandas as pd
import eval_probes_on_dataset
import jupyter_slack

In [2]:
def setup_args_and_folder(): 
    CONFIG_FILE = 'example/config/bert_ptb3.yaml'
    EXPERIMENT_NAME = ''
    SEED = 123

    class Object(object):
        pass

    cli_args = Object()
    cli_args.experiment_config = CONFIG_FILE
    cli_args.results_dir = EXPERIMENT_NAME
    cli_args.train_probe = -1
    cli_args.report_results = 1
    cli_args.seed = SEED

    yaml_args = yaml.load(open(cli_args.experiment_config))
    setup_new_experiment_dir(cli_args, yaml_args, cli_args.results_dir)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    yaml_args['device'] = device
    yaml_args['model_type'] = 'large'
    return yaml_args

yaml_args = setup_args_and_folder()

Constructing new results directory at results/BERT-disk-parse-distance-2020-4-27-15-43-4-797576/


/home/mli/anaconda3/envs/data-mining/lib/python3.6/site-packages/ipykernel_launcher.py:16: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  app.launch_new_instance()


# Requirements before running this code
- Corpus (conllx, for BERT)
    - Run `convert_splits_to_depparse.sh` (to get .conllx format)
    - Run `convert_conll_to_raw.py` (to get into raw text) 
- BERT-layer embeddings (.bert-layers.hdf5)
    - Run `convert_raw_to_bert.py` (Uses BERT to create bert-embeddings for ALL layers)
- Depth & Distance Params Path
    - Pretrained from data
- Ground Truths
    - Trees
        - Run Stanford CoreNLP's `ReadSentimentDataset` to get the ground truth trees
    - Sentiment
        - Run `apply_splits.py` to average all phrases' sentiments in the sentences

# Notes for generating the Tree data for SST
- This is for reading in the Ground Truth trees that is already given to us in SST
- We'll use Stanford's CoreNLP tools
- Run the ReadSentimentDataset `java -mx4g edu.stanford.nlp.sentiment.ReadSentimentDataset -inputDir data/SST-2/original -outputDir tmp/`
  - The ground truth already does subword partitions, so need to account for that

In [3]:
from nltk.tree import Tree

from reporter import WordPairReporter, WordReporter, prims_matrix_to_edges
from tqdm import tqdm
import copy

In [4]:
def read_trees(path):
    with open(path) as f:
        tree_lines = f.readlines()
        
    trees = [Tree.fromstring(treeline) for treeline in tree_lines]
    return trees

def read_sentiment_sentences(path):
    with open(path) as f:
        all_sentences = f.readlines()
        
        sentences, labels = [], []
        for pair in all_sentences:
            sentence, label = pair.split('\t')
            sentences.append(sentence)
            labels.append(label)
        return sentences, labels

In [5]:
data_base = Path('../../../data/SST-2')

train_path = data_base / 'sentence_splits' / 'train.tsv'
dev_path = data_base / 'sentence_splits' / 'dev.tsv'

# read in SST dataset
sst_trees_base = data_base / 'tree_format/'
gt_tree_train_path = sst_trees_base / 'train.txt'
gt_tree_dev_path = sst_trees_base / 'dev.txt'

train_sentiment, train_labels = read_sentiment_sentences(train_path)
dev_sentiment, dev_labels = read_sentiment_sentences(dev_path)

# Read into NTLK Trees
gt_train_trees = read_trees(gt_tree_train_path)
gt_dev_trees = read_trees(gt_tree_dev_path)

In [6]:
# Calculate distance between the two trees
# one tree's format is from structural probes
# other tree's format is from the dataset (gt_*_trees)
# import importlib
# importlib.reload(eval_probes_on_dataset)

word_dists, word_depths, predicted_edges, model, tokenizer = \
    eval_probes_on_dataset.report_on_stdin(yaml_args, dev_sentiment)

[demoing]: 1it [00:00,  5.36it/s]

Constructing TwoWordPSDProbe
Constructing OneWordPSDProbe


[demoing]: 1101it [00:20, 54.65it/s]


In [7]:
sigmoid = torch.nn.Sigmoid()
for line in tqdm(dev_sentiment, desc='Eval Dev Sentiment'):
    _, tokens_tensor, segments_tensors = eval_probes_on_dataset.prepare_sentence_for_bert(line, tokenizer)

    device = yaml_args['device']
    tokens_tensor = tokens_tensor.to(device)
    segments_tensors = segments_tensors.to(device)
    with torch.no_grad():
        logits, encoded_layers = model(tokens_tensor, segments_tensors)
        probs = sigmoid(logits)
        print(probs)

Eval Dev Sentiment:   1%|          | 8/1101 [00:00<00:43, 24.87it/s]

tensor([[0.4746]], device='cuda:0')
tensor([[0.5248]], device='cuda:0')
tensor([[0.4600]], device='cuda:0')
tensor([[0.4785]], device='cuda:0')
tensor([[0.4828]], device='cuda:0')
tensor([[0.4615]], device='cuda:0')
tensor([[0.4691]], device='cuda:0')
tensor([[0.4730]], device='cuda:0')
tensor([[0.4641]], device='cuda:0')
tensor([[0.5003]], device='cuda:0')
tensor([[0.4621]], device='cuda:0')


Eval Dev Sentiment:   2%|▏         | 19/1101 [00:00<00:32, 33.34it/s]

tensor([[0.6691]], device='cuda:0')
tensor([[0.5860]], device='cuda:0')
tensor([[0.4528]], device='cuda:0')
tensor([[0.4631]], device='cuda:0')
tensor([[0.4628]], device='cuda:0')
tensor([[0.4797]], device='cuda:0')
tensor([[0.5255]], device='cuda:0')
tensor([[0.4637]], device='cuda:0')
tensor([[0.4604]], device='cuda:0')
tensor([[0.4659]], device='cuda:0')
tensor([[0.4623]], device='cuda:0')


Eval Dev Sentiment:   3%|▎         | 31/1101 [00:00<00:25, 41.89it/s]

...

tensor([[0.4609]], device='cuda:0')
tensor([[0.4683]], device='cuda:0')
tensor([[0.4682]], device='cuda:0')
tensor([[0.4626]], device='cuda:0')
tensor([[0.4762]], device='cuda:0')
tensor([[0.5480]], device='cuda:0')
tensor([[0.5765]], device='cuda:0')
tensor([[0.5111]], device='cuda:0')
tensor([[0.5293]], device='cuda:0')
tensor([[0.4807]], device='cuda:0')
tensor([[0.4465]], device='cuda:0')
tensor([[0.4707]], device='cuda:0')


Eval Dev Sentiment: 100%|█████████▉| 1096/1101 [00:19<00:00, 58.38it/s]

tensor([[0.4797]], device='cuda:0')
tensor([[0.4618]], device='cuda:0')
tensor([[0.4723]], device='cuda:0')
tensor([[0.5783]], device='cuda:0')
tensor([[0.4715]], device='cuda:0')
tensor([[0.4696]], device='cuda:0')
tensor([[0.4611]], device='cuda:0')
tensor([[0.4803]], device='cuda:0')
tensor([[0.4760]], device='cuda:0')
tensor([[0.4730]], device='cuda:0')
tensor([[0.5222]], device='cuda:0')
tensor([[0.4703]], device='cuda:0')


Eval Dev Sentiment: 100%|██████████| 1101/1101 [00:19<00:00, 56.97it/s]

tensor([[0.4660]], device='cuda:0')


In [8]:
def earliest_root(a, b):
    for i in range(min(len(a), len(b))):
        if a[i] != b[i]:
            return i
    return -1

def get_adj_matrix(cur_tree):
    num_words = len(cur_tree.leaves())
    all_paths = [cur_tree.leaf_treeposition(i) for i in range(num_words)]
    matrix = [[0] * num_words for _ in range(num_words)]

    for a_idx, a in enumerate(all_paths):
        for b_idx, b in enumerate(all_paths):
            if a_idx < b_idx:
                root = earliest_root(a, b)
                matrix[a_idx][b_idx] = len(a) + len(b) - 2 * root
                matrix[b_idx][a_idx] = len(a) + len(b) - 2 * root
    return matrix

In [9]:
def report_uuas(predicted_edges, trees, split_name):
    uspan_total = 0
    uspan_correct = 0
    total_sents = 0
    for predicted_edge, gt_tree in tqdm(zip(predicted_edges, trees), desc='[uuas,tikz]'):
        words = gt_tree.leaves()
        poses = gt_tree.leaves()
        gold_edges = get_adj_matrix(gt_tree)
        gold_edges = prims_matrix_to_edges(gold_edges, words, poses)
        pred_edges = predicted_edge

        gold_span = set([tuple(sorted(x)) for x in gold_edges])
        pred_span = set([tuple(sorted(x)) for x in pred_edges])
        uspan_correct += len(gold_span.intersection(pred_span))
        uspan_total += len(gold_edges)
        total_sents += 1

    uuas = uspan_correct / float(uspan_total)
    return uuas

uuas_score = report_uuas(predicted_edges, gt_dev_trees, 'dev')
print(uuas_score)

[uuas,tikz]: 1101it [00:01, 628.10it/s]

0.278865893580425


In [10]:
def get_sentiments():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-10-e7d3484c8d5a>, line 2)

In [ ]:
%%notify "result"
result = f"Finished SST: {uuas_score:.4f}"
print(result)